# Lesson 2 - Assignment

In this assignment, you will implement a Decision Tree algorithm from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the functions to calculate Gini Index.

In [3]:
# Calculate the Gini index for a split dataset
#
# groups - list of lists, each list is a group of rows
def entropy(groups, classes):

	# count all samples at split point
	# sum of all rows in all groups - or total number of rows
	n_instances = float(sum([len(group) for group in groups]))

	# sum weighted Gini index for each group
	entropy = 0.0

	# for each group (i.e. 0 and 1)
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * np.log2(p)
			
		# weight the group score by its relative size
		entropy += (1.0 - score) * (size / n_instances)
	return entropy


In [4]:
# Calculate the Gini index for a split dataset
#
# groups - list of lists, each list is a group of rows
def gini_index(groups, classes):

	# count all samples at split point
	# sum of all rows in all groups - or total number of rows
	n_instances = float(sum([len(group) for group in groups]))

	# sum weighted Gini index for each group
	gini = 0.0

	# for each group, 
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


Question 2.1: Write a method that splits the 

In [5]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

Question 2.2: Write a method that loops over the dataset, determine the groups of rows that belong to the right or left split, and calculates the gini_index

In [6]:
def get_split(dataset):
	# class_values is the list of classes in the dataset, i.e. 
	class_values = list(set(row[-1] for row in dataset))

	# initialize the best values
	b_index, b_value, b_score, b_groups = 999, 999, 999, None

	# for each attribute in the dataset
	for index in range(len(dataset[0])-1):
		# and for each row in the dataset
		for row in dataset:
			
			# get new split
			groups = test_split(index, row[index], dataset)

			# calculate the gini for it
			gini = gini_index(groups, class_values)

			print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}


Question 2.3: Repeat question 2.2 using entropy

In [7]:
def get_split_entropy(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            entropy = entropy(groups, class_values)
            if entropy < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

Question 3.1: Write a method that takes in a group of rows and determines the class they belongs to. It should return the most common output value for a list of rows.

In [8]:
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

Question 3.2: Write a method that recursively split the data.
The method takes in a node, max_depth, min_size, and depth. Initially, the method would be called by passing the rood node and depth of 1. Here are the steps to help you implement:

- First, we create two groups for the data split and delete any existing groups from the node. As rows are used, they are no longer needed.
- Second, check if rows should be in left or right group, and if so we create a terminal node using the records we have.
- Third, check if maximum depth is reached and if so we create a terminal node.
- Fourth, process left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
- Fifth, process the right side in a similar manner as left side, as we rise back up the constructed tree to the root.

In [9]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

Question 3.3: Write a method that builds the tree. The method creates an initial split to determine root node, and then calls the split method.

In [10]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

Question 3.4: Write a method that takes in a node and rows of data, and predicts the class associated with each row.

In [11]:
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

Question 4: Train a decision tree using the banknote_authentication data

In [ ]:
from random import seed
from random import randrange
from csv import reader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load a CSV file
def load_csv(filename):
    # we can just use the Pandas read_csv function to load the data
    return pd.read_csv(filename, header=None)
 
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)

train, test = train_test_split(dataset, test_size=0.33, shuffle=False)

# need to change between np and list()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


,0,1,2,3,4
919,0.89512,4.77380,-4.84310,-5.59090,1
920,-5.48080,8.18190,0.27818,-5.03230,1
921,-2.88330,1.77130,0.68946,-0.46380,1
922,-1.41740,-2.25350,1.51800,0.61981,1
923,0.42830,-0.94981,-1.07310,0.32110,1


In [ ]:
tree = build_tree(train, max_depth, min_size)
predictions = list()
for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)
               
print('Accuracy: %s' % accuracy_score([row[-1] for row in test], predictions))

[Bonus] Question 5: Train and evaluate an SKLEARN decision tree model, and compare the results to your model 

Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

In [ ]:
That exercise was much more complex than I anticipated. It took some time to figure out what is expected 
to provide, during some googling I found a very similar implementation of the decision tree algorithm, which is
almost identical. I have a way better understanding of how process of creating a decision tree. I will definitely try
  reimplementing the algorithm from scratch soon.